In [4]:
import pandas as pd

In [3]:
df = pd.read_csv('../../data/feed_actions_last_10.csv', index_col=None, sep=';')

In [5]:
df_feed_actions = df.drop('row_number', axis=1)

In [6]:
df_feed_actions

,user_id,post_id,action,time
0,200,5181,view,2022-01-02 09:44:32
1,200,5228,view,2022-01-02 09:44:18
2,200,5319,view,2022-01-02 09:43:55
3,200,5379,view,2022-01-02 09:43:47
4,200,5361,view,2022-01-02 09:43:02
...,...,...,...,...
1631929,168552,7094,like,2022-01-24 18:53:16
1631930,168552,6846,like,2022-01-24 18:52:46
1631931,168552,7097,like,2022-01-24 18:51:53
1631932,168552,7094,view,2022-01-24 18:51:46


Преобразуем дату и время в timestamp

In [7]:
df_feed_actions['time'] = pd.to_datetime(df['time']).astype(int) // 10**6
df_feed_actions

,user_id,post_id,action,time
0,200,5181,view,1641116672000
1,200,5228,view,1641116658000
2,200,5319,view,1641116635000
3,200,5379,view,1641116627000
4,200,5361,view,1641116582000
...,...,...,...,...
1631929,168552,7094,like,1643050396000
1631930,168552,6846,like,1643050366000
1631931,168552,7097,like,1643050313000
1631932,168552,7094,view,1643050306000


In [8]:
df_users = pd.read_csv('../../data/users.csv', index_col=None, sep=';')
df_users = df_users.rename(columns={'id': 'user_id'})
df_users

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads
...,...,...,...,...,...,...,...,...
163200,168548,0,36,Russia,Kaliningrad,4,Android,organic
163201,168549,0,18,Russia,Tula,2,Android,organic
163202,168550,1,41,Russia,Yekaterinburg,4,Android,organic
163203,168551,0,38,Russia,Moscow,3,iOS,organic


In [9]:
df_posts = pd.read_csv('../../data/posts.csv', index_col=None, sep=';')
df_posts = df_posts.rename(columns={'id': 'post_id'})
df_posts

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


Объединим таблицы feed_actions, users и posts

In [10]:
df_merged_1 = df_users.merge(df_feed_actions, on='user_id')

In [11]:
df_merged_1

,user_id,gender,age,country,city,exp_group,os,source,post_id,action,time
0,200,1,34,Russia,Degtyarsk,3,Android,ads,5181,view,1641116672000
1,200,1,34,Russia,Degtyarsk,3,Android,ads,5228,view,1641116658000
2,200,1,34,Russia,Degtyarsk,3,Android,ads,5319,view,1641116635000
3,200,1,34,Russia,Degtyarsk,3,Android,ads,5379,view,1641116627000
4,200,1,34,Russia,Degtyarsk,3,Android,ads,5361,view,1641116582000
...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,Russia,Ivanteyevka,4,Android,organic,7094,like,1643050396000
1631930,168552,1,16,Russia,Ivanteyevka,4,Android,organic,6846,like,1643050366000
1631931,168552,1,16,Russia,Ivanteyevka,4,Android,organic,7097,like,1643050313000
1631932,168552,1,16,Russia,Ivanteyevka,4,Android,organic,7094,view,1643050306000


In [12]:
df_merged_2 = df_merged_1.merge(df_posts, on='post_id', how='left')

In [13]:
target = df_merged_2['action']
target

0          view
1          view
2          view
3          view
4          view
           ... 
1631929    like
1631930    like
1631931    like
1631932    view
1631933    view
Name: action, Length: 1631934, dtype: object

In [8]:
df_2 = df_merged_2.drop('action', axis=1)
df_2['action'] = target
df_2

NameError: name 'df_merged_2' is not defined

Преобразуем категориальные колонки в числовые

In [15]:
df_2['os'] = df_2['os'].astype('category').cat.codes
df_2['source'] = df_2['source'].astype('category').cat.codes
df_2['city'] = df_2['city'].astype('category').cat.codes

df_2

,user_id,gender,age,country,city,exp_group,os,source,post_id,time,text,topic,action
0,200,1,34,Russia,651,3,0,0,5181,1641116672000,I saw this film when it was originally release...,movie,view
1,200,1,34,Russia,651,3,0,0,5228,1641116658000,Alien was excellent. Many writers tried to cop...,movie,view
2,200,1,34,Russia,651,3,0,0,5319,1641116635000,"This amusing, sometimes poignant look at the H...",movie,view
3,200,1,34,Russia,651,3,0,0,5379,1641116627000,"Im starting to think that theres a conspiracy,...",movie,view
4,200,1,34,Russia,651,3,0,0,5361,1641116582000,"More wide-eyed, hysterical 50s hyper-cheerfuln...",movie,view
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,Russia,1061,4,0,1,7094,1643050396000,From the opening scenes of FIERCE PEOPLE (an i...,movie,like
1631930,168552,1,16,Russia,1061,4,0,1,6846,1643050366000,I have never seen a Barbara Steele movie that ...,movie,like
1631931,168552,1,16,Russia,1061,4,0,1,7097,1643050313000,Original Claymation Rudolph: Pretty good. Orig...,movie,like
1631932,168552,1,16,Russia,1061,4,0,1,7094,1643050306000,From the opening scenes of FIERCE PEOPLE (an i...,movie,view


Проверим данные на пропуски

In [16]:
if df_2.isna().any().any():
    print('есть пропуски')
else:
    print('пропусков нет')

пропусков нет


Преобразуем колонки 'topic' и 'country' в числовые, причем по популярности

In [17]:
# Вычисление частотности каждой категории
topic_frequency_map = df_2['topic'].value_counts().to_dict()

topic_frequency_map

{'movie': 1571980,
 'covid': 56752,
 'tech': 1613,
 'sport': 990,
 'politics': 372,
 'entertainment': 154,
 'business': 73}

In [18]:
for top_freq, topic in enumerate(topic_frequency_map.keys()):
    topic_frequency_map[topic] = top_freq

In [19]:
topic_frequency_map

{'movie': 0,
 'covid': 1,
 'tech': 2,
 'sport': 3,
 'politics': 4,
 'entertainment': 5,
 'business': 6}

Преобразование категориальной колонки 'topic' в числовую по частотности


In [20]:
df_2['topic_num'] = df_2['topic'].map(topic_frequency_map)
df_2

,user_id,gender,age,country,city,exp_group,os,source,post_id,time,text,topic,action,topic_num
0,200,1,34,Russia,651,3,0,0,5181,1641116672000,I saw this film when it was originally release...,movie,view,0
1,200,1,34,Russia,651,3,0,0,5228,1641116658000,Alien was excellent. Many writers tried to cop...,movie,view,0
2,200,1,34,Russia,651,3,0,0,5319,1641116635000,"This amusing, sometimes poignant look at the H...",movie,view,0
3,200,1,34,Russia,651,3,0,0,5379,1641116627000,"Im starting to think that theres a conspiracy,...",movie,view,0
4,200,1,34,Russia,651,3,0,0,5361,1641116582000,"More wide-eyed, hysterical 50s hyper-cheerfuln...",movie,view,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,Russia,1061,4,0,1,7094,1643050396000,From the opening scenes of FIERCE PEOPLE (an i...,movie,like,0
1631930,168552,1,16,Russia,1061,4,0,1,6846,1643050366000,I have never seen a Barbara Steele movie that ...,movie,like,0
1631931,168552,1,16,Russia,1061,4,0,1,7097,1643050313000,Original Claymation Rudolph: Pretty good. Orig...,movie,like,0
1631932,168552,1,16,Russia,1061,4,0,1,7094,1643050306000,From the opening scenes of FIERCE PEOPLE (an i...,movie,view,0


In [21]:
# Вычисление частотности по странам
country_frequency_map = df_2['country'].value_counts().to_dict()

for top_freq, country in enumerate(country_frequency_map.keys()):
    country_frequency_map[country] = top_freq

In [22]:
country_frequency_map

{'Russia': 0,
 'Ukraine': 1,
 'Belarus': 2,
 'Kazakhstan': 3,
 'Turkey': 4,
 'Finland': 5,
 'Azerbaijan': 6,
 'Estonia': 7,
 'Latvia': 8,
 'Cyprus': 9,
 'Switzerland': 10}

In [23]:
df_2['country_num'] = df_2['country'].map(country_frequency_map)

In [24]:
df_2['country'] = df_2['country_num']
df_2['topic'] = df_2['topic_num']

df_2.drop(['country_num','topic_num'], axis=1, inplace=True)

In [25]:
df_2

,user_id,gender,age,country,city,exp_group,os,source,post_id,time,text,topic,action
0,200,1,34,0,651,3,0,0,5181,1641116672000,I saw this film when it was originally release...,0,view
1,200,1,34,0,651,3,0,0,5228,1641116658000,Alien was excellent. Many writers tried to cop...,0,view
2,200,1,34,0,651,3,0,0,5319,1641116635000,"This amusing, sometimes poignant look at the H...",0,view
3,200,1,34,0,651,3,0,0,5379,1641116627000,"Im starting to think that theres a conspiracy,...",0,view
4,200,1,34,0,651,3,0,0,5361,1641116582000,"More wide-eyed, hysterical 50s hyper-cheerfuln...",0,view
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,1643050396000,From the opening scenes of FIERCE PEOPLE (an i...,0,like
1631930,168552,1,16,0,1061,4,0,1,6846,1643050366000,I have never seen a Barbara Steele movie that ...,0,like
1631931,168552,1,16,0,1061,4,0,1,7097,1643050313000,Original Claymation Rudolph: Pretty good. Orig...,0,like
1631932,168552,1,16,0,1061,4,0,1,7094,1643050306000,From the opening scenes of FIERCE PEOPLE (an i...,0,view


Преобразуем таргетную колонку 'action' в числовую: view -> 0, like -> 1 (успех)

In [26]:
df_2['action'].unique()

array(['view', 'like'], dtype=object)

In [27]:
df_2['action'] = df_2['action'].replace(['view', 'like'], [0, 1])

/var/folders/3y/rd5ld8cn7bv7yr9nlqd8vf0c0000gn/T/ipykernel_17516/526187676.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2['action'] = df_2['action'].replace(['view', 'like'], [0, 1])


In [28]:
df_2

,user_id,gender,age,country,city,exp_group,os,source,post_id,time,text,topic,action
0,200,1,34,0,651,3,0,0,5181,1641116672000,I saw this film when it was originally release...,0,0
1,200,1,34,0,651,3,0,0,5228,1641116658000,Alien was excellent. Many writers tried to cop...,0,0
2,200,1,34,0,651,3,0,0,5319,1641116635000,"This amusing, sometimes poignant look at the H...",0,0
3,200,1,34,0,651,3,0,0,5379,1641116627000,"Im starting to think that theres a conspiracy,...",0,0
4,200,1,34,0,651,3,0,0,5361,1641116582000,"More wide-eyed, hysterical 50s hyper-cheerfuln...",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,1643050396000,From the opening scenes of FIERCE PEOPLE (an i...,0,1
1631930,168552,1,16,0,1061,4,0,1,6846,1643050366000,I have never seen a Barbara Steele movie that ...,0,1
1631931,168552,1,16,0,1061,4,0,1,7097,1643050313000,Original Claymation Rudolph: Pretty good. Orig...,0,1
1631932,168552,1,16,0,1061,4,0,1,7094,1643050306000,From the opening scenes of FIERCE PEOPLE (an i...,0,0


Стемминг текста

In [29]:
from nltk.stem import PorterStemmer

# Создание экземпляра стеммера Портера из NLTK
stemmer = PorterStemmer()

# Определение функции для стемминга текста
def stem_text(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [30]:
df_text = df_2['text']

df_stemmed_text = df_text.apply(stem_text)

In [31]:
df_stemmed_text

0          i saw thi film when it wa origin releas in 198...
1          alien wa excellent. mani writer tri to copi it...
2          thi amusing, sometim poignant look at the holl...
3          im start to think that there a conspiracy, all...
4          more wide-eyed, hyster 50 hyper-ch that give n...
                                 ...                        
1631929    from the open scene of fierc peopl (an interpl...
1631930    i have never seen a barbara steel movi that i ...
1631931    origin claymat rudolph: pretti good. origin fr...
1631932    from the open scene of fierc peopl (an interpl...
1631933    journalist bob woodward blistering, scattersho...
Name: text, Length: 1631934, dtype: object

In [32]:
df_2['stemmed_text'] = df_stemmed_text
df_2

,user_id,gender,age,country,city,exp_group,os,source,post_id,time,text,topic,action,stemmed_text
0,200,1,34,0,651,3,0,0,5181,1641116672000,I saw this film when it was originally release...,0,0,i saw thi film when it wa origin releas in 198...
1,200,1,34,0,651,3,0,0,5228,1641116658000,Alien was excellent. Many writers tried to cop...,0,0,alien wa excellent. mani writer tri to copi it...
2,200,1,34,0,651,3,0,0,5319,1641116635000,"This amusing, sometimes poignant look at the H...",0,0,"thi amusing, sometim poignant look at the holl..."
3,200,1,34,0,651,3,0,0,5379,1641116627000,"Im starting to think that theres a conspiracy,...",0,0,"im start to think that there a conspiracy, all..."
4,200,1,34,0,651,3,0,0,5361,1641116582000,"More wide-eyed, hysterical 50s hyper-cheerfuln...",0,0,"more wide-eyed, hyster 50 hyper-ch that give n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,1643050396000,From the opening scenes of FIERCE PEOPLE (an i...,0,1,from the open scene of fierc peopl (an interpl...
1631930,168552,1,16,0,1061,4,0,1,6846,1643050366000,I have never seen a Barbara Steele movie that ...,0,1,i have never seen a barbara steel movi that i ...
1631931,168552,1,16,0,1061,4,0,1,7097,1643050313000,Original Claymation Rudolph: Pretty good. Orig...,0,1,origin claymat rudolph: pretti good. origin fr...
1631932,168552,1,16,0,1061,4,0,1,7094,1643050306000,From the opening scenes of FIERCE PEOPLE (an i...,0,0,from the open scene of fierc peopl (an interpl...


In [40]:
df_2.to_csv('../../data/data_preprocessed_with_text.csv', index=False, sep=';')

In [45]:
df_text_len = df_2['text'].map(len)
df_2['text_len'] = df_text_len

In [46]:
df_3 = df_2.drop(['text', 'stemmed_text'], axis=1)
df_3

,user_id,gender,age,country,city,exp_group,os,source,post_id,time,topic,action,text_len
0,200,1,34,0,651,3,0,0,5181,1641116672000,0,0,669
1,200,1,34,0,651,3,0,0,5228,1641116658000,0,0,663
2,200,1,34,0,651,3,0,0,5319,1641116635000,0,0,2399
3,200,1,34,0,651,3,0,0,5379,1641116627000,0,0,2813
4,200,1,34,0,651,3,0,0,5361,1641116582000,0,0,4091
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,1643050396000,0,1,2876
1631930,168552,1,16,0,1061,4,0,1,6846,1643050366000,0,1,1418
1631931,168552,1,16,0,1061,4,0,1,7097,1643050313000,0,1,876
1631932,168552,1,16,0,1061,4,0,1,7094,1643050306000,0,0,2876


In [ ]:
target = df_3['action']
df_3 = df_3.drop('action', axis=1)
df_3['action'] = target
df_3

,user_id,gender,age,country,city,exp_group,os,source,post_id,time,topic,text_len,action
0,200,1,34,0,651,3,0,0,5181,1641116672000,0,669,0
1,200,1,34,0,651,3,0,0,5228,1641116658000,0,663,0
2,200,1,34,0,651,3,0,0,5319,1641116635000,0,2399,0
3,200,1,34,0,651,3,0,0,5379,1641116627000,0,2813,0
4,200,1,34,0,651,3,0,0,5361,1641116582000,0,4091,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,1643050396000,0,2876,1
1631930,168552,1,16,0,1061,4,0,1,6846,1643050366000,0,1418,1
1631931,168552,1,16,0,1061,4,0,1,7097,1643050313000,0,876,1
1631932,168552,1,16,0,1061,4,0,1,7094,1643050306000,0,2876,0


In [13]:
df_3.to_csv('../../data/data_preprocessed.csv', index=False, sep=';')

In [11]:
df = pd.read_csv('../../data/data_preprocessed.csv', index_col=None, sep=';')

In [17]:
df_full = pd.read_csv('../../data/data_preprocessed_with_text.csv', index_col=None, sep=';')
df_full

,user_id,gender,age,country,city,exp_group,os,source,post_id,time,topic,action,stemmed_text
0,200,1,34,0,651,3,0,0,5181,1641116672000,0,0,i saw thi film when it wa origin releas in 198...
1,200,1,34,0,651,3,0,0,5228,1641116658000,0,0,alien wa excellent. mani writer tri to copi it...
2,200,1,34,0,651,3,0,0,5319,1641116635000,0,0,"thi amusing, sometim poignant look at the holl..."
3,200,1,34,0,651,3,0,0,5379,1641116627000,0,0,"im start to think that there a conspiracy, all..."
4,200,1,34,0,651,3,0,0,5361,1641116582000,0,0,"more wide-eyed, hyster 50 hyper-ch that give n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,1643050396000,0,1,from the open scene of fierc peopl (an interpl...
1631930,168552,1,16,0,1061,4,0,1,6846,1643050366000,0,1,i have never seen a barbara steel movi that i ...
1631931,168552,1,16,0,1061,4,0,1,7097,1643050313000,0,1,origin claymat rudolph: pretti good. origin fr...
1631932,168552,1,16,0,1061,4,0,1,7094,1643050306000,0,0,from the open scene of fierc peopl (an interpl...


In [18]:
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    line = line.replace('\n\n', ' ').replace('\n', ' ')
    return line

vectorizer = TfidfVectorizer(
    max_df=0.9, 
    min_df=0.003, 
    stop_words='english',
    preprocessor=preprocessing
)
tfidf_matrix = vectorizer.fit_transform(df_full['stemmed_text'])

In [19]:
tfidf_matrix

<1631934x4244 sparse matrix of type '<class 'numpy.float64'>'
	with 118255790 stored elements in Compressed Sparse Row format>

In [20]:
df_tfidf = df_full.copy()
df_tfidf['tfidf_sum'] = pd.DataFrame(tfidf_matrix.todense()).sum(axis=1)
df_tfidf['tfidf_mean'] = pd.DataFrame(tfidf_matrix.todense()).mean(axis=1)
df_tfidf['tfidf_max'] = pd.DataFrame(tfidf_matrix.todense()).max(axis=1)
df_tfidf

,user_id,gender,age,country,city,exp_group,os,source,post_id,time,topic,action,stemmed_text,tfidf_sum,tfidf_mean,tfidf_max
0,200,1,34,0,651,3,0,0,5181,1641116672000,0,0,i saw thi film when it wa origin releas in 198...,6.315479,0.001488,0.375393
1,200,1,34,0,651,3,0,0,5228,1641116658000,0,0,alien wa excellent. mani writer tri to copi it...,5.638878,0.001329,0.491565
2,200,1,34,0,651,3,0,0,5319,1641116635000,0,0,"thi amusing, sometim poignant look at the holl...",10.482928,0.002470,0.262795
3,200,1,34,0,651,3,0,0,5379,1641116627000,0,0,"im start to think that there a conspiracy, all...",11.189221,0.002636,0.194437
4,200,1,34,0,651,3,0,0,5361,1641116582000,0,0,"more wide-eyed, hyster 50 hyper-ch that give n...",12.599472,0.002969,0.266056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,1643050396000,0,1,from the open scene of fierc peopl (an interpl...,11.687211,0.002754,0.320822
1631930,168552,1,16,0,1061,4,0,1,6846,1643050366000,0,1,i have never seen a barbara steel movi that i ...,9.575619,0.002256,0.269894
1631931,168552,1,16,0,1061,4,0,1,7097,1643050313000,0,1,origin claymat rudolph: pretti good. origin fr...,6.197824,0.001460,0.477146
1631932,168552,1,16,0,1061,4,0,1,7094,1643050306000,0,0,from the open scene of fierc peopl (an interpl...,11.687211,0.002754,0.320822


In [23]:
df_tfidf.drop('stemmed_text', axis=1, inplace=True)
df_tfidf

In [25]:
df_tfidf.to_csv('../../data/data_preprocessed_tfidf.csv', index=False, sep=';')

In [115]:
df_users_likes = df_tfidf.groupby('user_id', as_index=False).agg({'action': 'sum'})
df_users_likes.rename(columns={'action': 'user_likes'}, inplace=True)
df_users_likes

,user_id,user_likes
0,200,3
1,201,2
2,202,4
3,203,2
4,204,3
...,...,...
163200,168548,1
163201,168549,0
163202,168550,0
163203,168551,1


In [114]:
df_posts_liked = df_tfidf.groupby('post_id', as_index=False).agg({'action': 'sum'})
df_posts_liked.rename(columns={'action': 'post_likes'}, inplace=True)
df_posts_liked

,post_id,post_likes
0,135,0
1,166,1
2,173,1
3,200,0
4,242,0
...,...,...
5030,7315,395
5031,7316,91
5032,7317,75
5033,7318,128


In [123]:
df_posts_liked[df_posts_liked['post_likes'] >= 100]

,post_id,post_likes
1082,2854,108
1107,2893,109
1150,2959,105
1156,2968,203
1169,2986,102
...,...,...
5023,7308,156
5024,7309,143
5027,7312,147
5030,7315,395


In [102]:
df_tfidf_grouped = df_tfidf[df_tfidf['action'] == 1].groupby(['user_id', 'topic'], as_index=False).agg({'action': 'count'})
df_tfidf_grouped

,user_id,topic,action
0,200,0,3
1,201,0,2
2,202,1,4
3,203,0,2
4,204,0,3
...,...,...,...
146585,168546,0,2
146586,168547,0,2
146587,168548,0,1
146588,168551,0,1


In [112]:
max_indexes = df_tfidf_grouped.groupby('user_id')['action'].idxmax()
df_favourite_topic = df_tfidf_grouped.loc[max_indexes][['user_id', 'topic']]
df_favourite_topic.rename(columns={'topic': 'topic_favourite'}, inplace=True)
df_favourite_topic

,user_id,topic_favourite
0,200,0
1,201,0
2,202,1
3,203,0
4,204,0
...,...,...
146585,168546,0
146586,168547,0
146587,168548,0
146588,168551,0


In [125]:
df_new = df_tfidf.drop('time', axis=1)
df_new

,user_id,gender,age,country,city,exp_group,os,source,post_id,topic,action,tfidf_sum,tfidf_mean,tfidf_max
0,200,1,34,0,651,3,0,0,5181,0,0,6.315479,0.001488,0.375393
1,200,1,34,0,651,3,0,0,5228,0,0,5.638878,0.001329,0.491565
2,200,1,34,0,651,3,0,0,5319,0,0,10.482928,0.002470,0.262795
3,200,1,34,0,651,3,0,0,5379,0,0,11.189221,0.002636,0.194437
4,200,1,34,0,651,3,0,0,5361,0,0,12.599472,0.002969,0.266056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,0,1,11.687211,0.002754,0.320822
1631930,168552,1,16,0,1061,4,0,1,6846,0,1,9.575619,0.002256,0.269894
1631931,168552,1,16,0,1061,4,0,1,7097,0,1,6.197824,0.001460,0.477146
1631932,168552,1,16,0,1061,4,0,1,7094,0,0,11.687211,0.002754,0.320822


In [128]:
df_new = df_new.merge(df_users_likes, on='user_id', how='left')

,user_id,gender,age,country,city,exp_group,os,source,post_id,topic,action,tfidf_sum,tfidf_mean,tfidf_max,user_likes_x,user_likes_y,user_likes,post_likes
0,200,1,34,0,651,3,0,0,5181,0,0,6.315479,0.001488,0.375393,3,3,3,107
1,200,1,34,0,651,3,0,0,5228,0,0,5.638878,0.001329,0.491565,3,3,3,43
2,200,1,34,0,651,3,0,0,5319,0,0,10.482928,0.002470,0.262795,3,3,3,236
3,200,1,34,0,651,3,0,0,5379,0,0,11.189221,0.002636,0.194437,3,3,3,11
4,200,1,34,0,651,3,0,0,5361,0,0,12.599472,0.002969,0.266056,3,3,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,0,1,11.687211,0.002754,0.320822,3,3,3,630
1631930,168552,1,16,0,1061,4,0,1,6846,0,1,9.575619,0.002256,0.269894,3,3,3,2380
1631931,168552,1,16,0,1061,4,0,1,7097,0,1,6.197824,0.001460,0.477146,3,3,3,1964
1631932,168552,1,16,0,1061,4,0,1,7094,0,0,11.687211,0.002754,0.320822,3,3,3,630


In [ ]:
df_new = df_new.merge(df_posts_liked, on='post_id', how='left')
df_new

In [131]:
df_new

,user_id,gender,age,country,city,exp_group,os,source,post_id,topic,action,tfidf_sum,tfidf_mean,tfidf_max,user_likes,post_likes
0,200,1,34,0,651,3,0,0,5181,0,0,6.315479,0.001488,0.375393,3,107
1,200,1,34,0,651,3,0,0,5228,0,0,5.638878,0.001329,0.491565,3,43
2,200,1,34,0,651,3,0,0,5319,0,0,10.482928,0.002470,0.262795,3,236
3,200,1,34,0,651,3,0,0,5379,0,0,11.189221,0.002636,0.194437,3,11
4,200,1,34,0,651,3,0,0,5361,0,0,12.599472,0.002969,0.266056,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,0,1,11.687211,0.002754,0.320822,3,630
1631930,168552,1,16,0,1061,4,0,1,6846,0,1,9.575619,0.002256,0.269894,3,2380
1631931,168552,1,16,0,1061,4,0,1,7097,0,1,6.197824,0.001460,0.477146,3,1964
1631932,168552,1,16,0,1061,4,0,1,7094,0,0,11.687211,0.002754,0.320822,3,630


In [132]:
df_new = df_new.merge(df_favourite_topic, on='user_id', how='left')
df_new

,user_id,gender,age,country,city,exp_group,os,source,post_id,topic,action,tfidf_sum,tfidf_mean,tfidf_max,user_likes,post_likes,topic_favourite
0,200,1,34,0,651,3,0,0,5181,0,0,6.315479,0.001488,0.375393,3,107,0.0
1,200,1,34,0,651,3,0,0,5228,0,0,5.638878,0.001329,0.491565,3,43,0.0
2,200,1,34,0,651,3,0,0,5319,0,0,10.482928,0.002470,0.262795,3,236,0.0
3,200,1,34,0,651,3,0,0,5379,0,0,11.189221,0.002636,0.194437,3,11,0.0
4,200,1,34,0,651,3,0,0,5361,0,0,12.599472,0.002969,0.266056,3,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,0,1,11.687211,0.002754,0.320822,3,630,0.0
1631930,168552,1,16,0,1061,4,0,1,6846,0,1,9.575619,0.002256,0.269894,3,2380,0.0
1631931,168552,1,16,0,1061,4,0,1,7097,0,1,6.197824,0.001460,0.477146,3,1964,0.0
1631932,168552,1,16,0,1061,4,0,1,7094,0,0,11.687211,0.002754,0.320822,3,630,0.0


In [139]:
df_new['topic_favourite'] = df_new['topic_favourite'].fillna(0)

,user_id,gender,age,country,city,exp_group,os,source,post_id,topic,action,tfidf_sum,tfidf_mean,tfidf_max,user_likes,post_likes,topic_favourite
0,200,1,34,0,651,3,0,0,5181,0,0,6.315479,0.001488,0.375393,3,107,0.0
1,200,1,34,0,651,3,0,0,5228,0,0,5.638878,0.001329,0.491565,3,43,0.0
2,200,1,34,0,651,3,0,0,5319,0,0,10.482928,0.002470,0.262795,3,236,0.0
3,200,1,34,0,651,3,0,0,5379,0,0,11.189221,0.002636,0.194437,3,11,0.0
4,200,1,34,0,651,3,0,0,5361,0,0,12.599472,0.002969,0.266056,3,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,0,1,11.687211,0.002754,0.320822,3,630,0.0
1631930,168552,1,16,0,1061,4,0,1,6846,0,1,9.575619,0.002256,0.269894,3,2380,0.0
1631931,168552,1,16,0,1061,4,0,1,7097,0,1,6.197824,0.001460,0.477146,3,1964,0.0
1631932,168552,1,16,0,1061,4,0,1,7094,0,0,11.687211,0.002754,0.320822,3,630,0.0


In [140]:
df_new['topic_favourite'] = df_new['topic_favourite'].astype(int)
df_new

,user_id,gender,age,country,city,exp_group,os,source,post_id,topic,action,tfidf_sum,tfidf_mean,tfidf_max,user_likes,post_likes,topic_favourite
0,200,1,34,0,651,3,0,0,5181,0,0,6.315479,0.001488,0.375393,3,107,0
1,200,1,34,0,651,3,0,0,5228,0,0,5.638878,0.001329,0.491565,3,43,0
2,200,1,34,0,651,3,0,0,5319,0,0,10.482928,0.002470,0.262795,3,236,0
3,200,1,34,0,651,3,0,0,5379,0,0,11.189221,0.002636,0.194437,3,11,0
4,200,1,34,0,651,3,0,0,5361,0,0,12.599472,0.002969,0.266056,3,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,0,1,11.687211,0.002754,0.320822,3,630,0
1631930,168552,1,16,0,1061,4,0,1,6846,0,1,9.575619,0.002256,0.269894,3,2380,0
1631931,168552,1,16,0,1061,4,0,1,7097,0,1,6.197824,0.001460,0.477146,3,1964,0
1631932,168552,1,16,0,1061,4,0,1,7094,0,0,11.687211,0.002754,0.320822,3,630,0


In [144]:
df_new.to_csv('../../data/data_preprocessed_new.csv', index=False, sep=';')
df_new

,user_id,gender,age,country,city,exp_group,os,source,post_id,topic,action,tfidf_sum,tfidf_mean,tfidf_max,user_likes,post_likes,topic_favourite
0,200,1,34,0,651,3,0,0,5181,0,0,6.315479,0.001488,0.375393,3,107,0
1,200,1,34,0,651,3,0,0,5228,0,0,5.638878,0.001329,0.491565,3,43,0
2,200,1,34,0,651,3,0,0,5319,0,0,10.482928,0.002470,0.262795,3,236,0
3,200,1,34,0,651,3,0,0,5379,0,0,11.189221,0.002636,0.194437,3,11,0
4,200,1,34,0,651,3,0,0,5361,0,0,12.599472,0.002969,0.266056,3,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631929,168552,1,16,0,1061,4,0,1,7094,0,1,11.687211,0.002754,0.320822,3,630,0
1631930,168552,1,16,0,1061,4,0,1,6846,0,1,9.575619,0.002256,0.269894,3,2380,0
1631931,168552,1,16,0,1061,4,0,1,7097,0,1,6.197824,0.001460,0.477146,3,1964,0
1631932,168552,1,16,0,1061,4,0,1,7094,0,0,11.687211,0.002754,0.320822,3,630,0
